In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import re
from typing import List, Any
from evolutionary.evolution_base import SolutionCandidate
from evolutionary.image_base import ImageSolutionData
from evolutionary.plotting import plot_fitness_statistics
from diffusers.utils import logging
from PIL import Image

In [2]:
RESULTS_FOLDER = "results"

# TODO these functions shall be extracted somewhere

def save_images_from_generation(population: List[SolutionCandidate[Any, ImageSolutionData]], generation: int):
    generation_dir = os.path.join(RESULTS_FOLDER, f"{generation}")
    os.makedirs(generation_dir, exist_ok=True)

    for index, candidate in enumerate(population):
        image_solution_data = candidate.result
        fitness_str = f"{candidate.fitness:.3f}"

        for i, image in enumerate(image_solution_data.images):
            image_name = f"{index}_{i}_fitness_{fitness_str}.png"
            image_path = os.path.join(generation_dir, image_name)
            image.save(image_path)

def parse_fitness_from_filename(filename):
    match = re.search(r"fitness_([0-9\.]+)\.png", filename)
    return float(match.group(1)) if match else 0

def create_generation_image_grid(generation, images_per_row=5):
    generation_dir = os.path.join(RESULTS_FOLDER, f"{generation}")
    
    # Delete plot if already exists, or else will conflict with the plot
    output_filepath = os.path.join(generation_dir, f"grid_{generation}.png")
    if os.path.exists(output_filepath):
        os.remove(output_filepath)
    
    image_files = glob.glob(os.path.join(generation_dir, "*.png"))
    image_files.sort(key=parse_fitness_from_filename, reverse=True)

    nrows = (len(image_files) + images_per_row - 1) // images_per_row
    fig, axes = plt.subplots(nrows=nrows, ncols=images_per_row, figsize=(15, 3 * nrows)) 

    for i, img_file in enumerate(image_files):
        ax = axes[i // images_per_row, i % images_per_row] if nrows > 1 else axes[i % images_per_row]
        img = mpimg.imread(img_file)
        ax.imshow(img)
        ax.axis('off')
        fitness = parse_fitness_from_filename(img_file)
        ax.set_title(f"{fitness:.3f}", fontsize=16)  # Fitness score as title

    # Turn off unused axes
    for ax in axes.flatten()[len(image_files):]:
        ax.axis('off')

    plt.suptitle(f"Generation {generation}", fontsize=20) 
    plt.tight_layout(pad=1.5)
    plt.savefig(output_filepath)
    plt.close(fig)
    return fig

def create_animation_from_generations(num_generations, output_file=os.path.join(RESULTS_FOLDER, "ga_evolution.gif"), time_per_frame=1000, time_last_frame=5000):
    frames = []
    for generation in range(num_generations):
        generation_path = os.path.join(RESULTS_FOLDER, f"{generation}", f"grid_{generation}.png")
        frames.append(Image.open(generation_path))
        
    durations = [time_per_frame] * (len(frames) - 1) + [time_last_frame]
    frames[0].save(output_file, save_all=True, append_images=frames[1:], duration=durations, loop=0)

In [3]:
from model_helpers.auto_pipeline import auto_diffusion_pipeline
logging.disable_progress_bar() # Or else your output will be full of progress bars
logging.set_verbosity_error() # Enable again if you are having problems
pipe = auto_diffusion_pipeline("stabilityai/sdxl-turbo")
pipe.set_progress_bar_config(disable=True)

Loaded StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.25.0",
  "_name_or_path": "stabilityai/sdxl-turbo",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "EulerAncestralDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}


In [ ]:
from evolutionary.prompt_encoding.image_creation import SDXLPromptEncodingImageCreator
from evolutionary.prompt_encoding.variation import PooledArithmeticCrossover, PooledUniformGaussianMutator, UniformGaussianMutatorArguments
from evolutionary.selectors import TournamentSelector
from evolutionary.algorithms.ga import GeneticAlgorithm
from evolutionary.image_evaluators import AestheticsImageEvaluator  

population_size = 10
num_generations = 100 

# Create the necessary components for the genetic algorithm
creator = SDXLPromptEncodingImageCreator(pipeline=pipe, batch_size=2, inference_steps=3)
evaluator = AestheticsImageEvaluator()  
crossover = PooledArithmeticCrossover(crossover_rate=0.5, crossover_rate_pooled=0.5)
mutation_arguments = UniformGaussianMutatorArguments(mutation_rate=0.1, mutation_strength=0.5, clamp_range=(-30, 30))
mutator = PooledUniformGaussianMutator(mutation_arguments, mutation_arguments)
selector = TournamentSelector(tournament_size=3)

# Prepare initial arguments
init_args = [creator.arguments_from_prompt(f"test cat {i}") for i in range(population_size)]

# Create and run the genetic algorithm
ga = GeneticAlgorithm(
    solution_creator=creator,
    evaluator=evaluator,
    mutator=mutator,
    crossover=crossover,
    selector=selector,
    population_size=population_size,
    num_generations=num_generations,
    initial_arguments=init_args,
    elitism_count=1,
    post_evaluation_callback=save_images_from_generation
)

best_solution = ga.run()
print(best_solution.fitness)
best_solution.result.images[0]

Model loaded successfully.


In [ ]:
for gen in range(num_generations):
    create_generation_image_grid(gen)

In [ ]:
create_animation_from_generations(num_generations)

![Animation Result](results/ga_evolution.gif)

In [ ]:

print(ga.best_fitness)
plot_fitness_statistics(ga)

## Saving a runs data to JSON

In [22]:
# todo